### Notebook Description
This notebook performs exploratory analysis of the synthetic shooting dataset.
It includes:
- structural checks
- shooter-level accuracy analysis
- heatmap of shot positions
- session-level accuracy trends
The results of this notebook are used for feature engineering and modeling.

# EDA for Synthetic Shooting Data

This notebook performs initial exploratory analysis of the synthetic dataset:
- basic data checks
- shooter/session accuracy analysis
- distance and weapon distribution
- shot heatmap for selected shooter
- accuracy trend visualization

In [3]:
import random
random.seed(42)
import pandas as pd
import numpy as np
np.random.seed(42)
import plotly.express as px
import plotly.graph_objects as go
import kaleido

pd.set_option("display.max_columns", 50)

from pathlib import Path
data_path = Path("..") / "data" / "shooting_synthetic.csv"
df = pd.read_csv(data_path)
df.head()

,shooter_id,session_id,series_id,shot_idx,x,y,distance_m,weapon,time_since_start,hit,timestamp
0,1,1,1,1,-5.821768,2.039715,50.0,carbine,1.103166,1,2024-01-08T12:00:01.103166
1,1,1,1,2,-5.320574,-0.646290,50.0,carbine,4.090732,1,2024-01-08T12:00:04.090732
2,1,1,1,3,-5.309397,2.602768,50.0,carbine,6.730863,1,2024-01-08T12:00:06.730863
3,1,1,1,4,-5.691640,1.250211,50.0,carbine,10.496486,1,2024-01-08T12:00:10.496486
4,1,1,1,5,-4.674707,0.979168,50.0,carbine,12.662518,1,2024-01-08T12:00:12.662518


## Basic dataset checks

In [4]:
print("Shape:", df.shape)
print("\nColumns:", df.columns.tolist())

print("\nInfo:")
df.info()

print("\nNull counts:")
print(df.isnull().sum())

print("\nUnique shooters:", df.shooter_id.nunique())
print("Unique sessions:", df.session_id.nunique())
print("Unique series:", df.series_id.nunique())

print("\nWeapon distribution:")
print(df.weapon.value_counts())

Shape: (50000, 11)

Columns: ['shooter_id', 'session_id', 'series_id', 'shot_idx', 'x', 'y', 'distance_m', 'weapon', 'time_since_start', 'hit', 'timestamp']

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   shooter_id        50000 non-null  int64  
 1   session_id        50000 non-null  int64  
 2   series_id         50000 non-null  int64  
 3   shot_idx          50000 non-null  int64  
 4   x                 50000 non-null  float64
 5   y                 50000 non-null  float64
 6   distance_m        50000 non-null  float64
 7   weapon            50000 non-null  object 
 8   time_since_start  50000 non-null  float64
 9   hit               50000 non-null  int64  
 10  timestamp         50000 non-null  object 
dtypes: float64(4), int64(5), object(2)
memory usage: 4.2+ MB

Null counts:
shooter_id          0
session_id        

In [5]:
print("Shots per shooter:")
display(df.groupby("shooter_id")["hit"].count())

print("Average distance per weapon:")
display(df.groupby("weapon")["distance_m"].mean())

pd.crosstab(df.weapon, df.distance_m)

Shots per shooter:


shooter_id
1     5000
2     5000
3     5000
4     5000
5     5000
6     5000
7     5000
8     5000
9     5000
10    5000
Name: hit, dtype: int64

Average distance per weapon:


weapon
carbine    161.871429
pistol      15.944291
rifle      330.309902
Name: distance_m, dtype: float64

distance_m,5.0,10.0,15.0,25.0,35.0,50.0,100.0,150.0,200.0,300.0,400.0,600.0,800.0
weapon,,,,,,,,,,,,,
carbine,0,0,0,0,0,2475,4490,4310,3470,1825,930,0,0
pistol,3405,3050,2640,1865,1115,580,0,0,0,0,0,0,0
rifle,0,0,0,0,0,0,1910,0,4910,5970,4140,1985,930


## Accuracy per shooter

In [6]:
# accuracy = average hit on all shots
acc_per_shooter = (
    df.groupby("shooter_id")["hit"]
    .mean()
    .reset_index()
    .rename(columns={"hit": "accuracy"})
)

acc_per_shooter

,shooter_id,accuracy
0,1,0.7378
1,2,0.5130
2,3,0.6990
3,4,0.5098
4,5,0.6102
5,6,0.5500
6,7,0.7982
7,8,0.9142
8,9,0.7520
9,10,0.4444


In [7]:
fig = px.bar(
    acc_per_shooter,
    x="shooter_id",
    y="accuracy",
    title="Accuracy per shooter",
    text="accuracy",
    range_y=[0, 1]
)
fig.update_traces(texttemplate="%{text:.2f}")
fig.show()

## Shot density heatmap for a selected shooter

In [8]:
shooter = 8
df_shooter = df[df.shooter_id == shooter]
df_shooter.head()

,shooter_id,session_id,series_id,shot_idx,x,y,distance_m,weapon,time_since_start,hit,timestamp
35000,8,1,1,1,-7.767611,2.927472,150.0,carbine,3.867355,1,2024-01-22T11:00:03.867355
35001,8,1,1,2,-1.520173,3.495774,150.0,carbine,5.781478,1,2024-01-22T11:00:05.781478
35002,8,1,1,3,-0.703928,1.253610,150.0,carbine,9.334625,1,2024-01-22T11:00:09.334625
35003,8,1,1,4,-3.706827,1.595419,150.0,carbine,11.541373,1,2024-01-22T11:00:11.541373
35004,8,1,1,5,-1.513193,-6.038151,150.0,carbine,14.549739,1,2024-01-22T11:00:14.549739


In [9]:
fig = px.density_heatmap(
    
    df_shooter,
    x="x",
    y="y",
    nbinsx=40,
    nbinsy=40,
    color_continuous_scale="Viridis",
    title=f"Shot Density Heatmap — Shooter {shooter}"
)

fig.update_layout(
    xaxis_title="X coordinate",
    yaxis_title="Y coordinate",
    xaxis=dict(range=[-50, 50]),
    yaxis=dict(range=[-50, 50]),
    width=700,
    height=600
)

fig.show()

## Accuracy per session per shooter

In [10]:
acc_session = (
    df.groupby(["shooter_id", "session_id"])["hit"]
    .mean()
    .reset_index()
    .rename(columns={"hit": "session_accuracy"})
)

shooter = 8
df_s = acc_session[acc_session.shooter_id == shooter]

fig = px.line(
    df_s,
    x="session_id",
    y="session_accuracy",
    
    markers=True,
    title=f"Session-level accuracy — Shooter {shooter}"
)
fig.update_traces(
    line=dict(width=2),
    marker=dict(size=5, opacity=0.85)
)

fig.update_layout(yaxis=dict(range=[0,1]))
fig.show()

## Session-level accuracy for all shooters

In [11]:
acc_session = (
    df.groupby(["shooter_id", "session_id"])["hit"]
    .mean()
    .reset_index()
    .rename(columns={"hit": "session_accuracy"})
)

fig = px.line(
    acc_session,
    x="session_id",
    y="session_accuracy",
    color="shooter_id",
    markers=True,
    title="Session-level accuracy across shooters"
)

fig.update_layout(
    yaxis=dict(range=[0, 1]),
    xaxis_title="Session",
    yaxis_title="Accuracy",
    legend_title="Shooter ID",
    width=950,
    height=600
)

fig.show()

In [12]:
!pip install kaleido

In [13]:
fig.write_image("../reports/figs/session_accuracy_all.png")

### Summary

- The synthetic dataset contains 50,000 shots.
- Shooter performance varies significantly due to individual skill and bias parameters.
- Shooter 8 demonstrates the highest accuracy (~0.9), shooter 10 the lowest (~0.45).
- Session-level trends show stable patterns for higher-skilled shooters and higher variance for lower-skilled ones.
- The dataset is ready for feature engineering and ML modeling.